# Run experiments with TraCi

In the previous tutorials, we learned how to run simulations with custom networks, and custom parameters. However, when a simulation is launched, it's not possible to interact with SUMO __during__ the execution. To overcome this problem, SUMO integrates a library named TraCi, allowing you to control simulations, simulation step by simulation step. At each step, it is therefore possible to modify the behavior of the infrastructures, for example by adapting them to the traffic, or to gather data for each simulation step.

In this tutorial, we'll learn how to use TraCi with sumo-experiments.

## The TraCiWrapper

The `run_traci` function from Experiment can only accept one TraCi function. As we said previously, TraCi functions are meant to change infrastructures behaviour, and to collect data. This can be a lot for a unique function, and it can be useful to split tis work in multiple functions. That's why the TraciWrapper was created. The aim of this class is to aggregate TraCi functions to make only one. In this class, TraCi functions are divided in two types :
- The behavioural functions are functions that modify the behaviour of the network. They return a new config dict, modified or not, to the TraCiWrapper that will use it for the next simulaton step.
- The stats functions are functions that gather data from the network. They return a dict where the key in the name of the data, and where the value is the data. The data can't be an iterable object. The idea of this implementation is to collect a dict with the same data names for each simulation step and to build a Pandas DataFrame a the end of the simulation.

Thus, there are two functions to add TraCi functions to the wrapper : `add_stats_function` to add a stats function, and `add_behavioural_function` to add a behavioural function. The aggregated function is named `final_function`.

***Note :** The TraciWrapper final function first executes the stats functions, and then the behavioural functions. It executes functions of each group __in the order in which they have been added__.*


## TraCi behavioural functions for pre-set networks

The pre-set networks from sumo-experiments implement behavioural functions. This is the main reason why detectors are also implemented. There are 3 different behavioural functions, that adapt traffic lights behaviour to detectors signals :

- The `boolean_detection` must be used with detectors generated by the `generate_boolean_detectors` method. When a traffic light detects a car on a lane where traffic light is red, and if it doesn't detect any car on the green lanes, it switches to the other phase if the current phase is set since more than the minimum duration time. If this condition doesn't occur, the traffic light switch to the other phase when the current phase last for more than the maximum duration time. This strategy needs three parameters : `min_duration_tl` is the minimum duration (in seconds) for a traffic light phase, `max_duration_tl` is the maximum duration (in seconds) for a traffic light phase, and `simulation_duration` is the total simulation duration.

- The `numerical_detection_all_vehicles` must be used with detectors generated by the `generate_numerical_detectors` method. When a traffic light detect more than the threshold of cars on a lane where traffic light is red, it switches to the other phase if the current phase is set since more than the minimum duration time. Both stopped and running cars are considered in this strategy. If this condition doesn't occur, the traffic light switch to the other phase when the current phase last for more than the maximum duration time. This strategy needs four parameters : `min_duration_tl` is the minimum duration for a traffic light phase, `max_duration_tl` is the maximum duration for a traffic light phase, `vehicle_threshold` is the number of vehicles detected required to switch to another phase and `simulation_duration` is the total simulation duration.

- The `numerical_detections_stopped_vehicles` must be used with detectors generated by the `generate_numerical_detectors` method. When a traffic light detect more than the threshold of cars on a lane where traffic light is red, it switches to the other phase if the current phase is set since more than the minimum duration time. Only stopped cars are considered in this strategy. If this condition doesn't occur, the traffic light switch to the other phase when the current phase last for more than the maximum duration time. This strategy needs four parameters : `min_duration_tl` is the minimum duration for a traffic light phase, `max_duration_tl` is the maximum duration for a traffic light phase, `vehicle_threshold` is the number of vehicles detected required to switch to another phase and `simulation_duration` is the total simulation duration.


## TraCi stats functions

Sumo-experiments implements stats functions that can be used for any network. This functions iterate over all vehicles or all lanes to get data from them. They can aggregat this data differently (sum, mean, max, min, etc) according to type of function you choose. All of this functions can be imported from the `traci_util` package.

## Example

Let's see a short example to illustrate all we said.

In [13]:
from sumo_experiments.preset_networks import SquareNetwork
from sumo_experiments import Experiment
from sumo_experiments.traci_util import *

# Pre-set network declaration
network = SquareNetwork()

# Experiment declaration
exp = Experiment(
    name = 'traci',
    infrastructures = network.generate_infrastructures,
    flows = network.generate_flows_all_directions,
    detectors = network.generate_boolean_detectors
)

# Experiment parameterization
exp.set_parameter('square_side_length', 3)
exp.set_parameter('min_duration_tl', 15)
exp.set_parameter('max_duration_tl', 60)
exp.set_parameter('yellow_time', 2)
exp.set_parameter('max_speed', 50)
exp.set_parameter('simulation_duration', 1000)
exp.set_parameter('flow_frequency', 25)
exp.set_parameter('stop_generation_time', 1000)
exp.set_parameter('distribution', 'binomial')
exp.set_parameter('lane_length', 100)
exp.set_parameter('boolean_detector_length', 10)

# TraCiWrapper declaration and adding functions
tw = TraciWrapper()
# Some stats functions
tw.add_stats_function(get_speed_data)
tw.add_stats_function(get_acceleration_data)
tw.add_stats_function(get_nb_vehicles)
# A behavioural function : boolean detection
tw.add_behavioural_function(network.boolean_detection)


# Runnin simulation with final simulation, and getting data
data = exp.run_traci(tw.final_function)

exp.clean_files()

Success.
{}
Success.
 Retrying in 1 seconds


/home/julesbompard/.local/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/julesbompard/.local/lib/python3.10/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


The data from the simulation are returned by the `run_traci` function, in a DataFrame object.

In [14]:
data

,simulation_step,mean_speed,mean_acceleration,nb_running_vehicles
0,1,NaN,NaN,0
1,2,NaN,NaN,0
2,3,NaN,NaN,0
3,4,NaN,NaN,0
4,5,NaN,NaN,0
...,...,...,...,...
995,996,7.239606,0.604184,7
996,997,6.878921,-0.360685,7
997,998,5.821858,-1.057062,7
998,999,5.032178,-0.789680,7


***Note :** The use of TraCi drastically decrease SUMO performances. We chose this way to collect data because of the size of self generated data files. Some files can weigh multiple GigaBytes, and are not in a table format. With our implementation, we can choose the data we want from the simulation, and they are returned directly in a table format, that can be plotted or stored easily.*

## Custom TraCi functions

You can create your own TraCi functions, stats or behavioural, for any SUMO network. We won't describe in this tutorial how to use the TraCi library from python, we highly recommand you to check the [documentation](https://sumo.dlr.de/docs/TraCI/Interfacing_TraCI_from_Python.html). Just remember that a function used by the TraCiWrapper must work for one and only one simulation step. Obviously, yan can also write functions that will overpass the TraCiWrapper, and in this case, they must work for the entire simulation.

**This is the end of this tutorial. If you have any remark or question, just send an email at jules.bompard.etu@univ-lille.fr .**